<a href="https://colab.research.google.com/github/dipanjanS/improving-RAG-systems-dhs2024/blob/main/Demo_3_Solutions_for_Wrong_Format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solutions for Wrong Format

Here we will explore the following strategies

- Native LLM Support
- Output Parsers


#### Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 832.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.

### Enter Open AI API Tokens

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

# Native LLM Output Response Support

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0,
                     model_kwargs={"response_format": {"type": "json_object"}})

In [ ]:
prompt = """Who won the Champions league in 2023,
            Output should be in JSON and have following fields:
            win_team, lose_team, venue, date, score
         """
response = chatgpt.invoke(prompt)

In [ ]:
print(response.content)

{
  "win_team": "Manchester City",
  "lose_team": "Inter Milan",
  "venue": "Atatürk Olympic Stadium",
  "date": "June 10, 2023",
  "score": "1-0"
}


In [ ]:
type(response.content)

str

# Output Parsers

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


# Define your desired data structure - like a python data class.
class GameDetails(BaseModel):
    win_team: str = Field(description="The winning team in the football game")
    lose_team: str = Field(description="The losing team in the football game")
    venue: str = Field(description="The venue of the football game")
    date: str = Field(description="The date of the football game")
    score: str = Field(description="The score of the football game")

parser = JsonOutputParser(pydantic_object=GameDetails)

In [ ]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"win_team": {"title": "Win Team", "description": "The winning team in the football game", "type": "string"}, "lose_team": {"title": "Lose Team", "description": "The losing team in the football game", "type": "string"}, "venue": {"title": "Venue", "description": "The venue of the football game", "type": "string"}, "date": {"title": "Date", "description": "The date of the football game", "type": "string"}, "score": {"title": "Score", "description": "The score of the football game", "type": "string"}}, "required": ["win_team", "lo

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_txt = """
             Who won the Champions league in 2023
             Use the following format when generating the output response

             Output format instructions:
             {format_instructions}`
             """

prompt = PromptTemplate.from_template(template=prompt_txt)

In [ ]:
llm_chain = (prompt
              |
            chatgpt
              |
            parser)

response = llm_chain.invoke({"format_instructions": parser.get_format_instructions()})

In [ ]:
response

{'win_team': 'Manchester City',
 'lose_team': 'Inter Milan',
 'venue': 'Atatürk Olympic Stadium',
 'date': '2023-06-10',
 'score': '1-0'}

In [ ]:
type(response)

dict